# LDA

In [1]:
import pandas as pd

ds = pd.read_parquet('./results/ds.parquet')
elapse_time = pd.read_csv('./results/elapse_time.csv')

In [2]:
from time import time

from gensim.corpora.dictionary import Dictionary
from gensim.models.ldamulticore import LdaMulticore
from gensim.models.coherencemodel import CoherenceModel
from utils import get_diversity, get_topics_lda
import optuna

def lda(docs):
    docs = docs.dropna()
    docs = docs[docs.apply(len) > 0]
    dictionary = Dictionary(docs)
    corpus = [dictionary.doc2bow(doc) for doc in docs]
    def objective(trial, get_lda=False):
        num_topics = trial.suggest_int('num_topics', 5, 100)
        alpha = trial.suggest_categorical('alpha_categorical', ['symmetric', 'asymmetric', 'scalar'])
        eta = trial.suggest_categorical('eta_categorical', ['symmetric', 'auto', 'scalar'])
        if alpha == 'scalar':
            alpha = trial.suggest_float('alpha', 0.01, 1)
        if eta == 'scalar':
            eta = trial.suggest_float('eta', 0.01, 1)
        model = LdaMulticore(
            corpus=corpus,
            id2word=dictionary,
            num_topics=num_topics,
            alpha=alpha,
            eta=eta,
        )
        if get_lda:
            return model
        c = CoherenceModel(model, texts=docs, dictionary=dictionary, coherence='c_v')
        cs = c.get_coherence()
        ts = get_diversity(get_topics_lda(model, dictionary))
        return cs * ts

    t_start = time()
    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=2) # FIXME: change n to 20

    best_trial = study.best_trial
    best_lda = objective(best_trial, get_lda=True)
    t_end = time()

    return {
        'model': best_lda,
        'time': t_end - t_start,
    }

In [3]:
def save_models_lda():
    r_time = []
    for v in ds:
        model = lda(ds[v])
        model['model'].save(f'./results/models_lda/{v}')
        r_time.append(model['time'])
    elapse_time['lda_training'] = pd.Series(r_time)

In [4]:
save_models_lda()

[I 2023-08-22 18:17:45,568] A new study created in memory with name: no-name-b36fdd6d-bf41-4123-bb33-2a65510a607b


[I 2023-08-22 18:18:01,347] Trial 0 finished with value: 0.31942839733113937 and parameters: {'num_topics': 77, 'alpha_categorical': 'symmetric', 'eta_categorical': 'scalar', 'eta': 0.46059631493874925}. Best is trial 0 with value: 0.31942839733113937.
[I 2023-08-22 18:18:14,042] Trial 1 finished with value: 0.3313815768612192 and parameters: {'num_topics': 46, 'alpha_categorical': 'asymmetric', 'eta_categorical': 'symmetric'}. Best is trial 1 with value: 0.3313815768612192.
[I 2023-08-22 18:18:19,022] A new study created in memory with name: no-name-6ec6d7a2-398e-48db-9f0c-20345cbce465
[I 2023-08-22 18:18:32,331] Trial 0 finished with value: 0.25505370192640536 and parameters: {'num_topics': 42, 'alpha_categorical': 'symmetric', 'eta_categorical': 'auto'}. Best is trial 0 with value: 0.25505370192640536.
[I 2023-08-22 18:18:47,633] Trial 1 finished with value: 0.23657885478529433 and parameters: {'num_topics': 65, 'alpha_categorical': 'symmetric', 'eta_categorical': 'auto'}. Best is t

In [5]:
elapse_time

,variant,nlp,preprocessing,lda_training
0,dLWN,33.653473,0.081993,33.434384
1,DLWN,33.653473,0.159020,33.695258


# BERTopic

In [6]:
from time import time

from bertopic import BERTopic
import pandas as pd

embedding_model = "paraphrase-multilingual-MiniLM-L12-v2"

def save_models_bertopic():
    path_prefix = './results/models_bertopic/'
    r_time = []
    for v in ds:
        t_start = time()
        docs = ds[v].dropna()
        docs = docs[docs.apply(len) > 0]
        docs = [' '.join(doc) for doc in docs]
        bertopic = BERTopic(language='multilingual', embedding_model=embedding_model)
        model = bertopic.fit(docs)
        t_end = time()
        model.save(
            f"{path_prefix}{v}",
            serialization="safetensors",
            save_embedding_model=embedding_model,
            save_ctfidf=True,
        )
        r_time.append(t_end - t_start)
    elapse_time['bertopic_training'] = pd.Series(r_time)

In [7]:
save_models_bertopic()

In [8]:
elapse_time

,variant,nlp,preprocessing,lda_training,bertopic_training
0,dLWN,33.653473,0.081993,33.434384,53.977166
1,DLWN,33.653473,0.159020,33.695258,13.361671


In [9]:
elapse_time.to_csv('./results/elapse_time.csv', index=False)